In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 43.4 MB/s eta 0:00:00


In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.2 MB/s eta 0:00:00


In [3]:
!pip install torch

In [4]:
from transformers import ElectraTokenizer, ElectraForMaskedLM, ElectraModel
import torch
from torch.nn.functional import cosine_similarity
import json
from collections import defaultdict, Counter
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import re

In [5]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator")
# model = ElectraForMaskedLM.from_pretrained("monologg/koelectra-base-v3-discriminator")
model = ElectraModel.from_pretrained("monologg/koelectra-base-v3-discriminator")

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ElectraModel(
  (embeddings): ElectraEmbeddings(
    (word_embeddings): Embedding(35000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): ElectraEncoder(
    (layer): ModuleList(
      (0-11): 12 x ElectraLayer(
        (attention): ElectraAttention(
          (self): ElectraSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): ElectraSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0

In [44]:
file_path = '/content/drive/MyDrive/Colab Notebooks/동일감정제거.jsonl'

In [45]:
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

In [38]:
emotion_labels = ['joy', 'anticipation', 'trust', 'surprise', 'disgust', 'fear', 'anger', 'sadness']

In [10]:
okt = Okt()
kkma = Kkma()
komoran = Komoran()

In [39]:
def stem_tokens(sentence, tokenizer):
    return tokenizer.morphs(sentence, stem=True)

In [32]:
sentence = "담주에  공지 뜨고 다담주에 티켓팅할 느낌"
tokens = stem_tokens(sentence, okt)
print(tokens)

[('담다', 'Verb'), ('공지', 'Noun'), ('뜨다', 'Verb'), ('다담', 'Noun'), ('주', 'Noun'), ('에', 'Josa'), ('티켓팅', 'Noun'), ('하다', 'Verb'), ('느낌', 'Noun')]


In [13]:
kkma.morphs(sentence)

['담', '주', '에', '공지', '뜨', '고', '다담', '주', '에', '티켓', '팅', '하', 'ㄹ', '느낌']

In [ ]:
komoran.morphs(sentence)

['담', '주', '에', '공지', '뜨', '고', '다', '담', '주', '에', '티켓팅할', '느낌']

In [30]:
okt.pos(sentence)

[('담주에', 'Verb'),
 ('공지', 'Noun'),
 ('뜨고', 'Verb'),
 ('다담', 'Noun'),
 ('주', 'Noun'),
 ('에', 'Josa'),
 ('티켓팅', 'Noun'),
 ('할', 'Verb'),
 ('느낌', 'Noun')]

In [15]:
def aggregate_tweets_by_emotion(data):
    emotion_tweets = defaultdict(list)
    emotion_unique_terms = defaultdict(set)
    all_words = set()

    for item in data:
        tweet = item['input']['form']
        terms = stem_tokens(tweet, okt)
        all_words.update(terms)
        for emotion in emotion_labels:
            if item['output'][emotion] == 'True':
                emotion_tweets[emotion].append(tweet)
                emotion_unique_terms[emotion].update(set(terms))

    return emotion_tweets, emotion_unique_terms, list(all_words)


def calculate_n(term, emotion_unique_terms):
    count = 0
    for unique_terms in emotion_unique_terms.values():
        if term in unique_terms:
            count += 1
    return count


def calculate_weighted_twf(emotion_tweets, emotion_unique_terms):
    emotion_lexicons = defaultdict(dict)
    total_documents = 8

    for emotion, tweets in emotion_tweets.items():
        term_count = Counter()
        tweet_count = Counter()
        total_terms = 0

        for tweet in tweets:
            terms = stem_tokens(tweet, okt)
            total_terms += len(terms)
            unique_terms = set(terms)
            for term in unique_terms:
                tweet_count[term] += 1
            for term in terms:
                term_count[term] += 1
        print(total_terms)
        print(tweet_count)



        for term, count in term_count.items():
            normalized_tweet_frequency = tweet_count[term] / total_terms
            n = calculate_n(term, emotion_unique_terms)

            if n < total_documents and n != 0:
                weight = 1 / n
            else:
                weight = 1 / 8

            weighted_twf = normalized_tweet_frequency * weight
            emotion_lexicons[emotion][term] = weighted_twf

    return emotion_lexicons

In [16]:
emotion_tweets, emotion_unique_terms, all_words = aggregate_tweets_by_emotion(data)

In [17]:
emotion_lexicons = calculate_weighted_twf(emotion_tweets, emotion_unique_terms)

369877
Counter({'&': 10043, 'others': 9712, '이': 7244, '하다': 6009, '너무': 4375, '에': 3683, '가': 3342, '좋다': 3232, '..': 3116, '진짜': 2817, '보다': 2800, '아': 2369, '도': 2213, '...': 2191, '거': 2041, '을': 1854, '귀엽다': 1804, '나': 1792, '있다': 1792, '들': 1765, '다': 1715, '는': 1688, '같다': 1663, '의': 1563, '로': 1229, '에서': 1209, '때': 1191, '아니다': 1167, '은': 1162, '를': 1102, '내': 1070, '으로': 1059, '.': 1010, '되다': 1010, '넘다': 1001, '웃기다': 964, '고': 951, '사랑': 946, '이다': 912, '만': 901, '안': 900, '하': 895, '들다': 883, '한': 872, '개': 855, '말': 852, '것': 828, '게': 826, '오늘': 798, '자다': 795, '사람': 785, '그': 783, '저': 783, '1': 772, '더': 772, '나오다': 736, '…': 733, '좋아하다': 728, '?': 723, '없다': 717, '마크': 709, '야': 702, '미치다': 689, '근데': 688, '오다': 677, '랑': 653, '생각': 632, '형': 622, '우리': 618, '해주다': 618, '보고': 604, '얼굴': 590, '또': 585, '가다': 582, '님': 574, '수': 571, '잘': 569, '(': 569, 'ㅠㅠ': 562, '하고': 558, '....': 556, '원': 551, '못': 550, '그렇다': 535, '않다': 528, '중': 528, '웃다': 524, ')': 523, '정말': 513,

In [20]:
for emotion, lexicons in emotion_lexicons.items():
    print(f"Emotion: {emotion}")
    print(sorted(lexicons.items(), key=lambda x: -x[1])[:50])
    print()


Emotion: joy
[('&', 0.003394033692281488), ('others', 0.00328217218156306), ('이', 0.002448111128834721), ('하다', 0.002030742652287112), ('너무', 0.0014785320525471982), ('에', 0.0012446705256071613), ('가', 0.0011294295130543396), ('좋다', 0.0010922549928760101), ('..', 0.0010530527715970444), ('진짜', 0.0009520056667486759), ('보다', 0.0009462605136302068), ('아', 0.0008006039845678428), ('도', 0.0007478837559513027), ('...', 0.0007404488519156368), ('거', 0.0006897563243997329), ('을', 0.0006265596400965726), ('귀엽다', 0.0006096621309246047), ('나', 0.0006056067287233324), ('있다', 0.0006056067287233324), ('들', 0.0005964820737704696), ('다', 0.0005795845645985017), ('는', 0.0005704599096456389), ('같다', 0.000562011155059655), ('의', 0.000528216136715719), ('로', 0.0004153407754469729), ('에서', 0.0004085817717781857), ('때', 0.00040249866847627726), ('아니다', 0.0003943878640737326), ('은', 0.0003926981131565358), ('를', 0.00037242110215017423), ('내', 0.0003616066962801147), ('으로', 0.0003578892442622818), ('.', 0.00

In [21]:
for emotion, lexicons in emotion_lexicons.items():
    print(f"Emotion: {emotion}")
    if '귀엽다' in lexicons:
        print(f"Score of '귀엽다': {lexicons['귀엽다']}")

Emotion: joy
Score of '귀엽다': 0.0006096621309246047
Emotion: surprise
Score of '귀엽다': 1.6609824157328256e-05
Emotion: anticipation
Score of '귀엽다': 6.738501737740684e-05
Emotion: trust
Score of '귀엽다': 0.00015780423568334675
Emotion: disgust
Score of '귀엽다': 5.1538217570567716e-05
Emotion: sadness
Score of '귀엽다': 2.583872501395291e-05
Emotion: anger
Score of '귀엽다': 3.241070849808777e-05
Emotion: fear
Score of '귀엽다': 1.5207737696940203e-05


In [22]:
def get_embeddings(words):
    inputs = tokenizer(words, return_tensors="pt", padding=True, truncation=True)
    inputs = {key: tensor.to(device) for key, tensor in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1)


def find_most_similar(word, all_words):
    word_embedding = get_embeddings([word])

    similarities = torch.nn.functional.cosine_similarity(word_embedding, all_embeddings, dim=-1)
    most_similar_idx = similarities.argmax().item()
    return all_words[most_similar_idx]

In [23]:
batch_size = 32
all_embeddings = []


for i in range(0, len(all_words), batch_size):
    batch_words = all_words[i: i + batch_size]
    batch_embeddings = get_embeddings(batch_words)
    all_embeddings.append(batch_embeddings)

all_embeddings = torch.cat(all_embeddings, dim=0)

In [24]:
def is_korean(word):
    return bool(re.match(r'^(##)?[\uac00-\ud7a3]+$', word))

In [25]:
def replace_word(sentence, target_form, emotion_lexicons, emotion, okt):
    words = stem_tokens(sentence, okt)

    candidates = []
    for word in words:
        if word != target_form and is_korean(word):
            weighted_twf = emotion_lexicons[emotion].get(word, 0)
            candidates.append((word, weighted_twf))

    candidates.sort(key=lambda x: x[1])

    word_to_replace = candidates[0][0] if candidates else None
    if word_to_replace is None:
        return sentence  # No candidates found, return original sentence

    most_similar_word = find_most_similar(word_to_replace, all_words)

    # Replace the stemmed word in the original word list
    replaced = False
    original_words = okt.morphs(sentence)

    # Keep track of the original sentence's spacing
    original_sentence_list = list(sentence)
    new_sentence_list = []

    start_index = 0
    for i, original_word in enumerate(original_words):
        end_index = start_index + len(original_word)
        segment = original_sentence_list[start_index:end_index]

        morphs_result = okt.morphs(original_word, stem=True)
        if morphs_result and morphs_result[0] == word_to_replace:
            segment = list(most_similar_word)
            replaced = True

        new_sentence_list.extend(segment)

        if end_index < len(sentence) and original_sentence_list[end_index] == ' ':
            new_sentence_list.append(' ')
            end_index += 1

        start_index = end_index

    # Reassemble the sentence if a word was replaced
    if replaced:
        new_sentence = "".join(new_sentence_list)
        return new_sentence
    else:
        return sentence  # No replacement was made

In [ ]:
augmented_data = []
for item in data:
    original_sentence = item['input']['form']
    target_form = item['input']['target']['form']
    emotion = next(e for e in emotion_labels if item['output'][e] == 'True') # Finding the emotion label
    try:
        new_sentence = replace_word(original_sentence, target_form, emotion_lexicons, emotion, okt)
        item['input']['form'] = new_sentence
    except IndexError:
        print(f"Error occurred with the original sentence: {original_sentence}")

    augmented_data.append(item)

In [ ]:
existing_data = []
with open('/content/drive/MyDrive/Colab Notebooks/동일감정제거.jsonl', 'r') as f:
    for line in f:
        existing_data.append(json.loads(line))

# augmented_data를 기존 데이터에 추가
existing_data.extend(augmented_data)

# 새로운 JSONL 파일에 저장
with open('동일감정제거_대체1.jsonl', 'w') as f:
    for entry in existing_data:
        f.write(json.dumps(entry, ensure_ascii=False))
        f.write('\n')


In [ ]:
existing_data[-1]

{'id': 'nikluge-2023-ea-train-038688',
 'input': {'form': '유방댕이...과일가게가면 어른들이 수박 뚜들기는거 보고 본인도 냅다 ㅠㅠㅠㅜㅡㅜㅠㅠㅠㅠㅠㅜㅡㅜㅠㅠㅏ악 두들기다가 한번쯤은 혼났을것 같아 &others&',
  'target': {'form': '유방댕이', 'begin': 0, 'end': 4}},
 'output': {'joy': 'False',
  'anticipation': 'True',
  'trust': 'False',
  'surprise': 'False',
  'disgust': 'False',
  'fear': 'False',
  'anger': 'False',
  'sadness': 'False'}}